# COMP30027 Machine Learning Project 2

## Description of text features

This notebook describes the pre-computed text features provided for Project 2. **You do not need to recompute the features yourself for this assignment** -- this information is just for your reference. However, feel free to experiment with different text features if you are interested. If you do want to try generating your own text features, some things to keep in mind:
- There are many different decisions you can make throughout the feature design process, from the text preprocessing to the size of the output vectors. There's no guarantee that the defaults we chose will produce the best possible text features for this classification task, so feel free to experiment with different settings.
- These features must be trained using a training corpus. Generally, the training corpus should not include validation samples, but for the purposes of this assignment we have used the entire non-test set (training+validation) as the training corpus, to allow you to experiment with different validation sets. If you recompute the text features as part of your own model, you should exclude validation samples and compute them on training samples only. For example, if you do N-fold cross-validation, this means generating N sets of features for N different training-validation splits.

In [2]:
import numpy as np
import pandas as pd

# read text
# for DEMONSTRATION PURPOSES, the entire training set will be used to train the models and also as a test set
x_train_original = pd.read_csv(r"book_rating_train.csv", index_col = False, delimiter = ',', header=0)
# use recipe name as an example
train_corpus_name = x_train_original['Name']
test_name = x_train_original['Name']
print(type( x_train_original))

<class 'pandas.core.frame.DataFrame'>


In [3]:
def count_rating(x_train_original):
    a = x_train_original.loc[x_train_original["rating_label"] == 3]# change to <= 5 later
    b = x_train_original.loc[x_train_original["rating_label"] == 4]
    c = x_train_original.loc[x_train_original["rating_label"] == 5]
    print("rating is 3: ", len(a))
    print("rating is 4: " , len(b))
    print("rating is 5: " , len(c))
    print("total is : " , len(a)+len(b)+len(c))


In [4]:
x_train_original.head
a = []
a = x_train_original.columns
index_dict = {}
for i in a:
    if(not x_train_original[pd.isnull(x_train_original[i])].index.empty):
        index_dict[i] = x_train_original[pd.isnull(x_train_original[i])].index

# len(x_train_original)
index_dict

a = pd.DataFrame(x_train_original["rating_label"].loc[index_dict["Publisher"]]) 
b = pd.DataFrame(x_train_original["rating_label"].loc[index_dict["Language"]]) 

count_rating(a)
count_rating(b)
# rating
pd.set_option('display.max_columns', None) # 显示所有列 
pd.set_option('display.max_rows', None) # 显示所有行 
pd.set_option('display.width', None) # 显示完整宽度 
pd.set_option('display.max_colwidth', -1) # 显示完整单元格内容
x_train_original["Description"].head(50)


rating is 3:  37
rating is 4:  100
rating is 5:  11
total is :  148
rating is 3:  4725
rating is 4:  11641
rating is 5:  836
total is :  17202


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12236\3283277286.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) # 显示完整单元格内容


0     Teachers will turn to this treasury of ideas again and again to build key language and literacy skills. Hands-on activities, games, songs and rhymes help kids develop vocabulary, letter knowledge, phonemic awareness, concepts of print, and more. Perfect for enriching reading and writing centers!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

测试数据是否对齐

In [5]:
length = len(x_train_original)
for i in range(length):
    print(len(x_train_original.iloc[i,:]))
    if (len(x_train_original.iloc[i,:]) != 10):
        # print(len((x_train_original.iloc[i,:])))
        print("ha")
    

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

忽略部分数据 没有label的直接删除

数据对其

## Count vectorizer

A count vectorizer converts documents to vectors which represent word counts. Each column in the output represents a different word and the values indicate the number of times that word appeared in the document. The overall size of a count vector matrix can be quite large (the number of columns is the total number of different words used across all documents in a corpus), but most entries in the matrix are zero (each document contains only a few of all the possible words). Therefore, it is most efficient to represent the count vectors as a sparse matrix.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# preprocess text and compute counts

vocab_name = CountVectorizer(stop_words='english').fit(train_corpus_name)

# generate counts for a new set of documents
x_train_name = vocab_name.transform(train_corpus_name)
x_test_name = vocab_name.transform(test_name)

# check the number of words in vocabulary
print(len(vocab_name.vocabulary_))
# check the shape of sparse matrix
print(x_train_name.shape)
x_train_name
vocab_name.vocabulary_

20766
(23063, 20766)


{'best': 2164,
 'dr': 5804,
 'jean': 9941,
 'reading': 15269,
 'writing': 20488,
 'dwell': 5979,
 'free': 7522,
 'boomer': 2544,
 'big': 2216,
 'surprise': 18114,
 'll': 11026,
 'annie': 1163,
 'dodge': 5689,
 'wauneka': 20103,
 'navajo': 12760,
 'leader': 10702,
 'activist': 652,
 'castles': 3285,
 'britain': 2763,
 'ireland': 9786,
 'ultimate': 19358,
 'reference': 15407,
 'book': 2530,
 '350': 393,
 'gazetteer': 7807,
 'entries': 6426,
 'cities': 3785,
 'ladies': 10534,
 'beguine': 2060,
 'communities': 4137,
 'medieval': 11865,
 'low': 11166,
 'countries': 4577,
 '1200': 37,
 '1565': 94,
 'tangi': 18332,
 'crystal': 4783,
 'snow': 17268,
 'fairy': 6881,
 'rainbow': 15157,
 'magic': 11338,
 'weather': 20126,
 'fairies': 6878,
 'road': 15860,
 'al': 861,
 'qaeda': 14998,
 'story': 17828,
 'bin': 2238,
 'laden': 10533,
 'right': 15801,
 'hand': 8484,
 'man': 11457,
 'gay': 7803,
 'lesbian': 10795,
 'guide': 8335,
 'college': 4025,
 'life': 10879,
 'mummy': 12566,
 'tomb': 18841,
 'com

## doc2vec

doc2vec methods are an extension of word2vec. word2vec maps words to a high-dimensional vector space in such a way that words which appear in similar contexts will be close together in the space. doc2vec does a similar embedding for multi-word passages. The doc2vec (or Paragraph Vector) method was introduced by:

**Le & Mikolov (2014)** Distributed Representations of Sentences and Documents<br>
https://arxiv.org/pdf/1405.4053v2.pdf

The implementation of doc2vec used for this project is from gensim and documented here:<br>
https://radimrehurek.com/gensim/models/doc2vec.html

The size of the output vector is a free parameter. Most implemementations use around 100-300 dimensions, but the best size depends on the problem you're trying to solve with the embeddings and the number of training samples, so you may wish to try different vector sizes. We provided doc2vec features for Name (vec_size = 100), Authors (vec_size = 20) and Description (vec_size = 100). The vectors themselves represent directions in a high-dimensional concept space; the columns do not represent specific words or phrases. Values in the vector are continuous real numbers and can be negative.

In [7]:
import gensim

# size of the output vector
vec_size = 100

# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    for i, line in enumerate(txt):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# tokenize a training corpus
corpus_name = list(tokenize_corpus(train_corpus_name))

# train doc2vec on the training corpus
model = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40)
model.build_vocab(corpus_name)
model.train(corpus_name, total_examples=model.corpus_count, epochs=model.epochs)

# tokenize new documents
doc = list(tokenize_corpus(test_name, tokens_only=True))

# generate embeddings for the new documents
x_test_name = np.zeros((len(doc),vec_size))
for i in range(len(doc)):
    x_test_name[i,:] = model.infer_vector(doc[i])
    
# check the shape of doc_emb
print(x_test_name.shape)

KeyboardInterrupt: 